In [1]:
# !pip install -q tensorflow#==2.9.3
# !pip install -q keras#==2.4.3
# !pip install -q scipy==1.8.1
# !pip install -q scikit-learn==1.1.1
# !pip install -q pandas#==1.0.1
# !pip install -q esprima
# !pip install -q numpy#==1.18.5
# !pip3 install -a scipy==1.7.1 numpy==1.18.5 scikit-learn==0.24.2 --no-cache-dir --no-binary :all:

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, classification_report, confusion_matrix
from keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback,CSVLogger
from keras.models import Sequential, load_model,Model , clone_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dense, Reshape, Input
from keras.preprocessing.text import Tokenizer
from keras.losses import BinaryCrossentropy
from keras.backend import clear_session
from keras.initializers import Constant
from threading import current_thread
from keras.utils import plot_model
from sklearn.utils import shuffle
from keras.optimizers import Adam
from keras.utils import Sequence
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from keras import backend as K
from numpy.linalg import norm
from functools import partial
from random import uniform
from numpy import asarray
from copy import deepcopy
import concurrent.futures
from keras import layers
import matplotlib as mpl
from keras import Model
from numpy import save
from numpy import load
import seaborn as sns
# import esprima as esp
from time import time
import pandas as pd
import numpy as np
import warnings
import tempfile
import logging
import urllib
import random
import pickle
import pydot
import torch
import math
import html
import nltk
import os
import re
import gc
warnings.filterwarnings("ignore")
np.random.seed(123)
plt.style.use('ggplot')
# mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
par_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Hiden"
model_info_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Results_TabuSearch/Model info"
results_path = "/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Results_TabuSearch" # images

In [5]:
class GarbageCollectorCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

In [6]:
def _replaceitem(x):
    if type(x) is list:
        if x[0]<0.5:return 0.0
    else:
        x=float(x)
        if x<0.5:return 0.0
    return 1.0

In [7]:
def save_object(obj, filename,path):
    """
    _ INPUT (obj) THE OBJECT WE NEED SAVW IT (filename) THE NAME OF OBJECT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'wb') as outp:
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
    outp.close()

In [8]:
def load_object(filename,path):
    """
    _ INPUT THE NAME OF OBJECT WE NEED LOAD IT
    """
    filename = os.path.join(path,filename)
    with open(filename+".pkl", 'rb') as outp:
        loaded_object = pickle.load(outp)
    outp.close()
    return loaded_object

In [9]:
def create_model(shape=None,without_first_layer=False):
    """
    _INPUT (without_first_layer) IF WE WANT TO CREAT MODEL WITHOUT FIRST LSTM LAYER (shape) IF THE PREVIOS PAREMETER IS TRUE WE NEED DIFINE THE SHAPE FOR INPUT LAYER
    _OUTPUT THE MODEL 
    """
    clear_session()
    model = Sequential() 
    name_model = 'tabu_search_model.best.hdf5'
    model.add(layers.InputLayer(input_shape=shape))
    # TextCNN with 4 conv layers
    model.add(Conv1D(128, 7, activation='tanh', input_shape=(None, 32)))
    model.add(Conv1D(128, 15, activation='tanh'))
    model.add(Conv1D(128, 25, activation='tanh'))
    model.add(Conv1D(128, 35, activation='tanh'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    # model.load_weights(os.path.join(model_info_path,name_model))
    plot_model(model, to_file=os.path.join(model_info_path,name_model[:-10]+'.png'), show_shapes=True, show_layer_names=True)
    # model.summary()
    return model

In [10]:
def plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,F1_test,Ac_test,F1_train,Ac_train,method_name,step=1):
  title = 'Our method Vs '+method_name
  fig, (ax1, ax2) = plt.subplots(1, 2, layout='constrained', figsize=(15, 4))
  
  fig.suptitle(title)
  
  ax1.plot([i for i, x in enumerate(Best_F1_test) if i % step == 0],[x for i, x in enumerate(Best_F1_test) if i % step == 0], color="red",label="F1_Score_test")
  ax1.plot([i for i, x in enumerate(Best_Ac_test) if i % step == 0],[x for i, x in enumerate(Best_Ac_test) if i % step == 0], color="blue",label="Accurce_test")
  ax1.plot([i for i, x in enumerate(Best_F1_train) if i % step == 0],[x for i, x in enumerate(Best_F1_train) if i % step == 0], color="yellow",label="F1_Score_train")
  ax1.plot([i for i, x in enumerate(Best_Ac_train) if i % step == 0],[x for i, x in enumerate(Best_Ac_train) if i % step == 0], color="green",label="Accurce_train")
  
  ax1.legend(loc="upper right")
  ax1.set_xlabel('Chunk')
  ax1.set_ylabel('Results per Chunk')

  ax2.plot([i for i, x in enumerate(F1_test) if i % step == 0],[x for i, x in enumerate(F1_test) if i % step == 0], color="red",label="F1_Score_test")
  ax2.plot([i for i, x in enumerate(Ac_test) if i % step == 0],[x for i, x in enumerate(Ac_test) if i % step == 0], color="blue",label="Accurce_test")
  ax2.plot([i for i, x in enumerate(F1_train) if i % step == 0],[x for i, x in enumerate(F1_train) if i % step == 0], color="yellow",label="F1_Score_train")
  ax2.plot([i for i, x in enumerate(Ac_train) if i % step == 0],[x for i, x in enumerate(Ac_train) if i % step == 0], color="green",label="Accurce_train")
  
  ax2.legend(loc="upper right")
  ax2.set_xlabel('Chunk')
  ax2.set_ylabel('Results per Chunk')  

  plt.savefig(os.path.join(os.path.join(results_path,'Imges'),title))
  plt.show()

In [11]:
def plot_results(chunk_number,f1=None,ac=None,f1_train=None,ac_train=None,step=5):

    plt.figure(figsize=(15, 5))
    
    plt.title("Tabu Search results for chunk number "+str(chunk_number))
    plt.plot([i for i, x in enumerate(f1) if i % step == 0],[x for i, x in enumerate(f1) if i % step == 0], color="red",label="F1_Score_test")
    plt.plot([i for i, x in enumerate(ac) if i % step == 0],[x for i, x in enumerate(ac) if i % step == 0], color="blue",label="Accurce_test")
    plt.plot([i for i, x in enumerate(f1_train) if i % step == 0],[x for i, x in enumerate(f1_train) if i % step == 0], color="yellow",label="F1_Score_train")
    plt.plot([i for i, x in enumerate(ac_train) if i % step == 0],[x for i, x in enumerate(ac_train) if i % step == 0], color="green",label="Accurce_train")

    plt.legend(loc="upper left")
    plt.xlabel('iteration')
    plt.ylabel('results per iteration')
    plt.xscale('log',base=2)

    plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Tabu Search results for chunk number "+str(chunk_number)))
    plt.show()

In [12]:
def plot_dimensional_reduction(tabu,softmax,single_agent,average):
  # set width of bar
  barWidth = 0.05
  fig = plt.subplots(figsize =(8, 5))
  # Set position of bar on X axis
  br1 = np.arange(len(tabu))
  br2 = [x + barWidth for x in br1]
  br3 = [x + barWidth for x in br2]
  br4 = [x + barWidth for x in br3]
  # Make the plot
  
  plt.bar(br1, tabu, color ='r', width = barWidth,edgecolor ='grey', label ='Tabu_Search')
  plt.bar(br2, softmax, color ='g', width = barWidth,edgecolor ='grey', label ='softmax')
  plt.bar(br3, single_agent, color ='b', width = barWidth,edgecolor ='grey', label ='single_agent')
  plt.bar(br4, average, color ='y', width = barWidth,edgecolor ='grey', label ='average')
  # Adding Xticks
  plt.xlabel('Chunk number', fontweight ='bold', fontsize = 10)
  plt.ylabel('The remaining dimensions after being reduced from 100', fontweight ='bold', fontsize = 8)
  plt.xticks([r + barWidth for r in range(len(number_of_chunk))],[str(i) for i,_ in enumerate(number_of_chunk)])
  plt.legend(loc="upper right")
  plt.savefig(os.path.join(os.path.join(results_path,'Imges'),"Dimensional reduction amount"))
  plt.show()

In [13]:
def PointsInCircum(r,n=100):
    return [(math.cos(2*math.pi/n*x)*r+np.random.normal(-30,30),math.sin(2*math.pi/n*x)*r+np.random.normal(-30,30)) for x in range(1,n+1)]

In [14]:
import seaborn as sns 
from sklearn.metrics import confusion_matrix, cohen_kappa_score,accuracy_score,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
import pandas as pd 

def plot_conf_matrix(true, pred, classes):
    cf = confusion_matrix(true, pred)
    plt.figure(figsize=(12,10))
    sns.set(font_scale=1.4)
    sns.heatmap(np.round(cf / cf.sum(axis=0), 3).T, annot=True, annot_kws={"size": 16},fmt='g',
                cmap='Blues', xticklabels = classes ,yticklabels = classes)
    plt.savefig('confusion_matrix.png')
    plt.savefig('confusion_matrix.svg')

    plt.show()
# plot_conf_matrix(labels, y_preds, class_names)

In [15]:
def get_callbacks():
  filepath = os.path.join(model_info_path,'Tabu_Search_.best.hdf5')
  history_logger = CSVLogger(os.path.join(model_info_path,'history1.csv'), separator=",", append=True)
  checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
  es = EarlyStopping(monitor='val_loss', patience=5)
  callbacks_list = [GarbageCollectorCallback(),es] # ,history_logger,checkpoint
  return callbacks_list

In [16]:
def evaluate_result(y_predict,y_true):
    y_predict[y_predict>0.5] = 1
    y_true = list(map(_replaceitem, y_true))
    y_predict = list(map(_replaceitem, y_predict))
    return f1_score(y_true=y_true,y_pred=y_predict),accuracy_score(y_true=y_true,y_pred=y_predict)

In [17]:
def chunking_data(NUMBER):
    """
    _INPUT (Data) CHUNK OF DATASET (test_size=0.20) SIZE OF SPLITNG
    _OUTPUT (X_train , y_train , X_test , y_test)
    """
    x_chunk , y_chunk = load_object('X_train_hiden_for_chunk_'+str(NUMBER),par_path) , load_object('y_train_hiden_for_chunk_'+str(NUMBER),par_path)
    X_new_chunks , y_new_chunks = np.array(np.split(x_chunk, x_chunk.shape[0]/100)) , np.array(np.split(y_chunk, y_chunk.shape[0]/100))
    return X_new_chunks[:-1] , y_new_chunks[:-1] , X_new_chunks[-1] , y_new_chunks[-1]

In [18]:
def apply_change_on_data(X,mask,change):
  if change[0] is None:
    return apply_mask(X,mask),mask
  mask[change[0]][change[1]] = 1-mask[change[0]][change[1]]
  return apply_mask(X.copy(),mask.copy()),mask.copy()

In [19]:
def neighbour_solution(solution):
  neighbour = solution.copy()
  x,y=np.random.randint(low=0, high=100,size=2)
  neighbour[x][y]=1-neighbour[x][y]
  return neighbour,x,y

In [20]:
def Tabu_list_checker(solution):
  for t in Tabu_list:
    if np.all(np.equal(solution,t)):
      return True
  return False

In [21]:
def check_aspiration_criteria(f1,ac):
  f1_aspiration_criteria,ac_aspiration_criteria= 0.90,0.90 #np.random.uniform(low=0.80,high=1.0,size=2)
  if(f1>=f1_aspiration_criteria)and(ac>=ac_aspiration_criteria):return True
  else:return False

In [22]:
def generate_neighbour_solution(solution):
  neighbour=np.random.randint(low=0,high=2,size=(number_of_calss,X_train.shape[2])) #,x,y= neighbour_solution(solution)
  while True:
    t = Tabu_list_checker(neighbour)
    if t:neighbour=np.random.randint(low=0,high=2,size=(number_of_calss,X_train.shape[2])) #,x,y
    else:break
  return neighbour#,x,y 

In [23]:
def get_similarity(X_test,X_train):
  similarites=[]
  for x in X_test:
    similarity=[]
    for X in X_train:
      similarity.append(np.mean(np.sum(x*X, axis=1)/(norm(x, axis=1)*norm(X, axis=1))))
    similarites.append(similarity.index(max(similarity)))
  return similarites

In [24]:
def update_tabu_list(solution):
  Tabu_list.append(solution)
  value_of_update = len(Tabu_list)-Tabu_tenur
  del Tabu_list[:value_of_update]
  return Tabu_list

In [25]:
def apply_mask(X,mask,cluster=None):
  if cluster is None:
    for i,x in enumerate(X):
      X[i] = x*mask
  else:
    for i,x in enumerate(X):
      X[i] = x*mask[cluster[i]]
  return X

In [26]:
def get_statistical_information(X):
  new_x = []
  for x in X:
    new_x.append(np.array([np.ndarray.min(x),np.ndarray.max(x),np.ndarray.mean(x),np.ndarray.std(x),np.median(x),np.var(x)]))
  return np.array(new_x)

In [27]:
def save(i):
  save_object(solutions, 'solutions_'+str(i),results_path)
  save_object(F1_train, 'F1_train_'+str(i),results_path)
  save_object(F1_test, 'F1_test_'+str(i),results_path)
  save_object(Ac_train, 'Ac_train_'+str(i),results_path)
  save_object(Ac_test, 'Ac_test_'+str(i),results_path)
  save_object(Tabu_list, 'Tabu_list_'+str(i),results_path)
  save_object(agglomerativeClustering, 'agglomerativeClustering',results_path)

In [28]:
def load(i):
  F1_train = load_object('F1_train_'+str(i),results_path)
  F1_test = load_object('F1_test_'+str(i),results_path)
  Ac_train = load_object('Ac_train_'+str(i),results_path)
  Ac_test = load_object('Ac_test_'+str(i),results_path)
  return F1_train,F1_test,Ac_train,Ac_test

In [29]:
from sklearn.cluster import KMeans,DBSCAN,AgglomerativeClustering
from sklearn.metrics import silhouette_score,calinski_harabasz_score,davies_bouldin_score
NUMBER = 0 
number_of_calss = 35
Tabu_tenur,Tabu_list=100,[]
number_of_chunk =[i[:-4] for i in os.listdir(par_path)[::-1] if i[:7]=='X_train']
number_of_chunk = number_of_chunk[:4]
agglomerativeClustering = AgglomerativeClustering(n_clusters=number_of_calss, affinity='euclidean') 

In [ ]:
for i,x in tqdm(enumerate(number_of_chunk)):
  if i!=3:continue
  X_train , y_train = load_object('X_train_hiden_for_chunk_'+str(i),par_path) , load_object('y_train_hiden_for_chunk_'+str(i),par_path)
  X_test , y_test = load_object('X_test_hiden_for_chunk_'+str(i),par_path) , load_object('y_test_hiden_for_chunk_'+str(i),par_path)
  number_of_iteration = X_train.shape[2]*10
  if i==0:
    model = create_model(shape=X_train[0].shape,without_first_layer=True)
    model.save(os.path.join(model_info_path,"tabu_search_model.h5"))
  else:
    Tabu_list = load_object('Tabu_list_'+str(i-1),results_path)
    agglomerativeClustering = load_object('agglomerativeClustering',results_path)
    model = load_model(os.path.join(model_info_path,"tabu_search_model.h5"))
  cluster_train = agglomerativeClustering.fit_predict(get_statistical_information(X_train))
  cluster_test = agglomerativeClustering.fit_predict(get_statistical_information(X_test))
  current_solution,solutions,F1_train,Ac_train,F1_test,Ac_test = np.random.randint(low=0,high=2,size=(number_of_calss,X_train.shape[2])),[],[],[],[],[]
  for step in tqdm(range(number_of_iteration)):
    x_train = X_train.copy()
    x_train = apply_mask(x_train,current_solution,cluster_train)
    history = model.fit(x_train, y_train,epochs=10,batch_size=100,validation_split=0.20,callbacks=get_callbacks(), verbose=0)
    x_test = X_test.copy()
    x_test = apply_mask(x_test,current_solution,cluster_test)
    f1_test,ac_test =  evaluate_result(model.predict(x_test, verbose=0),y_test)
    f1_train,ac_train =  evaluate_result(model.predict(x_train, verbose=0),y_train)

    Tabu_list = update_tabu_list(current_solution)
    aspiration_criteria = check_aspiration_criteria(f1_test,ac_test)
    if not aspiration_criteria:
      model = load_model(os.path.join(model_info_path,"tabu_search_model.h5"))
    else: 
      model.save(os.path.join(model_info_path,"tabu_search_model.h5"))
    solutions.append(current_solution)
    F1_train.append(f1_train)
    F1_test.append(f1_test)
    Ac_train.append(ac_train)
    Ac_test.append(ac_test)
    save(i)
    current_solution = generate_neighbour_solution(current_solution)

0it [00:00, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

## Final Results

In [ ]:
Basic_model = load_model(os.path.join("/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Results/Model info","model2.h5")) 
softmax_model = load_model(os.path.join("/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/softmax/Model info","model_with_dfs.h5")) 
single_agent_model = load_model(os.path.join("/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/single_agent/Model info","model_with_dfs.h5"))
average_model = load_model(os.path.join("/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/average/Model info","model_with_dfs.h5"))

In [ ]:
Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train = [],[],[],[]
Basic_F1_test,Basic_Ac_test,Basic_F1_train,Basic_Ac_train = [],[],[],[]
softmax_F1_test,softmax_Ac_test,softmax_F1_train,softmax_Ac_train = [],[],[],[]
single_agent_F1_test,single_agent_Ac_test,single_agent_F1_train,single_agent_Ac_train = [],[],[],[]
average_F1_test,average_Ac_test,average_F1_train,average_Ac_train = [],[],[],[]
_tabu,_softmax,_single_agent,_average = [],[],[],[]

In [ ]:
for i,_ in tqdm(enumerate(number_of_chunk)):
  X_train , y_train = load_object('X_train_hiden_for_chunk_'+str(i),par_path) , load_object('y_train_hiden_for_chunk_'+str(i),par_path)
  X_test , y_test = load_object('X_test_hiden_for_chunk_'+str(i),par_path) , load_object('y_test_hiden_for_chunk_'+str(i),par_path)
  F1_train,F1_test,Ac_train,Ac_test = load(i)
  solutions = load_object('solutions_'+str(i),results_path)
  solutions_of_softmax = load_object('results_of_softmax_'+str(i),"/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/softmax")
  solutions_of_single_agent = load_object('results_of_single_agent_'+str(i),"/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/single_agent")
  solutions_of_average = load_object('results_of_average_'+str(i),"/content/drive/MyDrive/Colab Notebooks/Muawiya/Js_Contana/On Payload File/Featuer_selection_results/average")

  f1_test,ac_test =  evaluate_result(Basic_model.predict(X_test, verbose=0),y_test)
  Basic_F1_test.append(f1_test)
  Basic_Ac_test.append(ac_test)
  
  f1_train,ac_train =  evaluate_result(Basic_model.predict(X_train, verbose=0),y_train)
  Basic_F1_train.append(f1_train)
  Basic_Ac_train.append(ac_train)

  f1_test,ac_test =  evaluate_result(softmax_model.predict(apply_mask(X_test.copy(),solutions_of_softmax), verbose=0),y_test)
  softmax_F1_test.append(f1_test)
  softmax_Ac_test.append(ac_test)
  
  f1_train,ac_train =  evaluate_result(softmax_model.predict(apply_mask(X_train.copy(),solutions_of_softmax), verbose=0),y_train)
  softmax_F1_train.append(f1_train)
  softmax_Ac_train.append(ac_train)
  
  f1_test,ac_test =  evaluate_result(single_agent_model.predict(apply_mask(X_test.copy(),solutions_of_single_agent), verbose=0),y_test)
  single_agent_F1_test.append(f1_test)
  single_agent_Ac_test.append(ac_test)
  
  f1_train,ac_train =  evaluate_result(single_agent_model.predict(apply_mask(X_train.copy(),solutions_of_single_agent), verbose=0),y_train)
  single_agent_F1_train.append(f1_train)
  single_agent_Ac_train.append(ac_train)

  f1_test,ac_test =  evaluate_result(average_model.predict(apply_mask(X_test.copy(),solutions_of_average), verbose=0),y_test)
  average_F1_test.append(f1_test)
  average_Ac_test.append(ac_test)

  f1_train,ac_train =  evaluate_result(average_model.predict(apply_mask(X_train.copy(),solutions_of_average), verbose=0),y_train)
  average_F1_train.append(f1_train)
  average_Ac_train.append(ac_train)

  f1_max = max(F1_test)
  temp = F1_test.index(f1_max)
  ac_max = Ac_test[temp] 
  Best_F1_test.append(f1_max)
  Best_Ac_test.append(ac_max)

  f1_max = max(F1_train)
  ac_max = Ac_train[F1_train.index(f1_max)]
  Best_F1_train.append(f1_max)
  Best_Ac_train.append(ac_max)  


  solution = solutions[temp]
  sums = []
  for s in solution:
    sums.append(sum(s))
  _tabu.append(np.average(sums))
  _softmax.append(sum(solutions_of_softmax))
  _single_agent.append(sum(solutions_of_single_agent))
  _average.append(sum(solutions_of_average))

  plot_results(i,f1=F1_test,ac=Ac_test,f1_train=F1_train,ac_train=Ac_train,step=10)

In [ ]:
save_object(_tabu, '_tabu',results_path)
save_object(_softmax, '_softmax',results_path)
save_object(_single_agent, '_single_agent',results_path)
save_object(_average, '_average',results_path)
plot_dimensional_reduction(_tabu,_softmax,_single_agent,_average)

plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,Basic_F1_test,Basic_Ac_test,Basic_F1_train,Basic_Ac_train,'Basic model')
plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,softmax_F1_test,softmax_Ac_test,softmax_F1_train,softmax_Ac_train,'softmax method')
plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,single_agent_F1_test,single_agent_Ac_test,single_agent_F1_train,single_agent_Ac_train,'single_agent method')
plot_finall_results(Best_F1_test,Best_Ac_test,Best_F1_train,Best_Ac_train,average_F1_test,average_Ac_test,average_F1_train,average_Ac_train,'average method')